# LSTM prediction - many to many
Inputs are 2 sequances (week_num and isHoliday) and output is the corresponding sequance of weekly_sales

### Imports and loading data

In [1]:
import numpy as np # linear algebra
import datetime # week number computation
from numpy import newaxis
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Bidirectional

from numpy import array
from numpy import hstack

from keras.models import Sequential
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

print ('import completed')

# Loading data
dataset = pd.read_csv('/home/lengyel/Desktop/kaggle dataset/walmart/train.csv', parse_dates=['Date'])
dataset = dataset[dataset.Dept==1][dataset.Store==1] #Selecting Department and Store No.1 to reduce complexity

week_num = [
    datetime.date(dataset['Date'][i].year, dataset['Date'][i].month, dataset['Date'][i].day).isocalendar()[1]
           for i in range(len(dataset))]
dataset['Week_num'] = week_num
dataset = dataset.drop(columns=['Store','Dept','Date']) # These columns are unnecessary due to selecting 
                                                    # a specific Store and a Department, and adding week_num

Using TensorFlow backend.


import completed


/home/lengyel/anaconda3/envs/MyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Prepare data

In [2]:
 # Map isHoliday True/False data to 0s and 1s
dataset['IsHoliday'] = dataset['IsHoliday'].astype(int)

In [3]:
#defining splitting ratios
splitrate_train = 0.6
splitrate_val = 1-splitrate_train/2

In [4]:
# splitting dataset to train and test datasets
# creating 2 input sequences

split_ix_train = int(len(dataset)*splitrate_train)
split_ix_val = int((len(dataset)-split_ix_train)/2)

train_in1 = array([dataset['Week_num'][i] for i in range(split_ix_train)]).astype(np.float64)
train_in2 = array([dataset['IsHoliday'][i] for i in range(split_ix_train)])
train_out = array([dataset['Weekly_Sales'][i] for i in range(split_ix_train)]).astype(np.float64)

test_in1 = array([dataset['Week_num'][i] for i in range(split_ix_train,len(dataset))]).astype(np.float64)
test_in2 = array([dataset['IsHoliday'][i] for i in range(split_ix_train,len(dataset))])
test_out = array([dataset['Weekly_Sales'][i] for i in range(split_ix_train,len(dataset))]).astype(np.float64)

In [ ]:
# Reshape datasets to fit LSTM input
train_in1 = train_in1.reshape(train_in1.shape[0], 1, 1)
train_in2 = train_in2.reshape(train_in2.shape[0], 1, 1)
train_out = train_out.reshape(train_out.shape[0], 1, 1)

test_in1 = test_in1.reshape(test_in1.shape[0], 1, 1)
test_in2 = test_in2.reshape(test_in2.shape[0], 1, 1)
test_out = test_out.reshape(test_out.shape[0], 1, 1)

In [ ]:
#concatenating 2 input sequances  to 1, both for training and validating
train_in = array([(train_in1[i, 0,0], train_in2[i, 0,0]) for i in range(train_in1.shape[0])])
test_in = array([(test_in1[i, 0,0], test_in2[i, 0,0]) for i in range(test_in1.shape[0])])

In [ ]:
# reshaping input and output sequances to fit LSTM 3D
train_in = train_in.reshape(train_in.shape[0], 1, train_in.shape[1])
test_in = test_in.reshape(test_in.shape[0], 1, test_in.shape[1])

train_out = train_out.reshape(train_out.shape[0],1, 1)
test_out = test_out.reshape(test_out.shape[0],1, 1)

# splitting test dataset to validation and test datasets
val_in = test_in[:,35:,:]
test_in = test_in[:,:35,:]

val_out = test_out[:,35:,:]
test_out = test_out[:,:35,:]

In [ ]:
# normalizing inputs 
scaler = MinMaxScaler(feature_range = (0, 1))

train_in_scaled, test_in_scaled = train_in.copy(), test_in.copy()

scaled_array=scaler.fit_transform(train_in[:,:,0])
train_in_scaled[:,:,0] = scaled_array.reshape(train_in[:,:,0].shape[0], 1)

scaled_array=scaler.fit_transform(test_in[:,:,0])
test_in_scaled[:,:,0] = scaled_array.reshape(test_in[:,:,0].shape[0], 1)

In [ ]:
#normalizing outputs

train_out_scaled = scaler.fit_transform(train_out[:,:,0])  
train_out_scaled = train_out_scaled.reshape(train_out_scaled.shape[0],1, 1)

test_out_scaled = scaler.fit_transform(test_out[:,:,0])  
test_out_scaled = test_out_scaled.reshape(test_out_scaled.shape[0],1, 1)

In [ ]:
#temp
val_in_scaled = test_in_scaled[:split_ix_val]
val_out_scaled = test_out_scaled[:split_ix_val]

test_in_scaled = test_in_scaled[split_ix_val:]
test_out_scaled = test_out_scaled[split_ix_val:]

### Define model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(7, activation='relu', return_sequences=True, batch_input_shape=(None,1, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

### Train model

In [ ]:
# fit model
history = model.fit(train_in_scaled, train_out_scaled, epochs=1100, verbose=0,
                    validation_data = (val_in_scaled, val_out_scaled))

### Evaluate results

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history

plt.figure(figsize=(13,5))
plt.plot(epoch_count, training_loss, 'ro', c='r')
plt.plot(epoch_count, test_loss, 'ro',c='b')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim(top=1, bottom=0)
plt.show();

In [ ]:
yhat = model.predict(test_in_scaled, verbose=0)

In [ ]:
plt.figure(figsize=(13,5))
plt.plot(test_out_scaled.reshape(test_out_scaled.shape[0]), 'ro', c='b')
plt.plot(yhat.reshape(yhat.shape[0]), 'ro', c='r')

plt.legend(['Actual value', 'Prediction'])
#plt.ylim(top=1, bottom=-0.3)
plt.show